In [2]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark
using LDLFactorizations

In [3]:
mutable struct mLCP
  c0      :: Float64          # constant term in objective
  q1      :: Vector           # c
  q2      :: Vector            # -b
  M11     :: SparseMatrixCSC
  M12     :: SparseMatrixCSC
  M21     :: SparseMatrixCSC     # A
  M22     :: SparseMatrixCSC
  lvar    :: Vector
  uvar    :: Vector
  ilow    :: Vector
  iupp    :: Vector
  irng    :: Vector
end


In [13]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i])/2
        end
    end

    return x_tilde
end
    

function starting_points(M11, M12, M21, q1, q2, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)

    tmp_diag = vcat(ρ*ones(n_cols),spzeros(n_rows))

    #J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag), :U))
    J_fact = lu(J_augm-Diagonal(tmp_diag))
    J_P = J_fact.p
    init_xλ = J_fact \ [zeros(n_cols) ; -q2]
    #init_xλ2 = J_fact \ [c ; zeros(n_rows)]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = M11*x0 - M12*λ0 + q1
    s0_l[ilow] = @views dual_val[ilow]
    s0_u[iupp] = @views -dual_val[iupp]

    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = @views max(-1.5*minimum(x0_l1 - lvar[ilow]), 1.e0)
        δs_l1 = @views max(-1.5*minimum(s0_l[ilow]), 1.e-4)
    end
    
    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = @views max(-1.5*minimum((uvar[iupp] - x0_u1)), 1.e0)
        δs_u1 = @views max(-1.5*minimum(s0_u[iupp]), 1.e-4)
    end
    
    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = @views s0_l[ilow] .+ δs_l1
    s0_u1 = @views s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = @views s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    if length(ilow) == 0
        δx_l2, δs_l2 = 0., 0.
    else
        δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
        δs_l2 = @views δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    end
    if length(iupp) == 0
        δx_u2, δs_u2 = 0., 0.
    else
        δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
        δs_u2 = @views δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
    end
    δx = max(δx_l2, δx_u2)
    δs = max(δs_l2, δs_u2)
    x0[ilow] .+= δx
    x0[iupp] .-= δx
    s0_l[ilow] = @views s0_l[ilow] .+ δs
    s0_u[iupp] = @views s0_u[iupp] .+ δs

    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert @views all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)

    return x0, λ0, s0_l, s0_u, J_P
end


starting_points (generic function with 1 method)

In [14]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end


function compute_μ(x_minus_lvar, uvar_minus_x, s_l, s_u, nb_low, nb_upp)
    return (s_l' * x_minus_lvar + s_u' * uvar_minus_x) / (nb_low + nb_upp)
end




function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [15]:
function solve_augmented_system_aff!(J_fact, Δ_aff, Δ_x_λ, r1, r2, x_minus_lvar, uvar_minus_x, 
                                     s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)
    
    F_x_λ_aff = [-r1 
                 -r2]
    F_x_λ_aff[ilow] += @views s_l[ilow]
    F_x_λ_aff[iupp] -= @views s_u[iupp]
    
    Δ_x_λ = ldiv!(Δ_x_λ , J_fact, F_x_λ_aff)
#     Δ_aff = @views [Δ_x_λ
#                     -s_l[ilow] - s_l[ilow].*Δ_x_λ[1:n_cols][ilow]./x_minus_lvar
#                     -s_u[iupp] + s_u[iupp].*Δ_x_λ[1:n_cols][iupp]./uvar_minus_x]
    Δ_aff[1:n_cols+n_rows] = Δ_x_λ
    Δ_aff[n_cols+n_rows+1:n_cols+n_rows+n_low] = @views -s_l[ilow] - s_l[ilow].*Δ_x_λ[1:n_cols][ilow]./x_minus_lvar
    Δ_aff[n_cols+n_rows+n_low+1:end] = @views -s_u[iupp] + s_u[iupp].*Δ_x_λ[1:n_cols][iupp]./uvar_minus_x
    return Δ_aff
end

function solve_augmented_system_cc!(J_fact, Δ_cc, Δ_x_λ ,Δ_aff, σ, μ,x_minus_lvar, uvar_minus_x, 
                                    s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)


    rxs_l = @views -σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
    rxs_u = @views σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]

    F_x_λ_cc = zeros(n_cols+n_rows)
    F_x_λ_cc[ilow] += rxs_l./x_minus_lvar
    F_x_λ_cc[iupp] += rxs_u./uvar_minus_x
    
    Δ_x_λ = ldiv!(Δ_x_λ , J_fact, F_x_λ_cc)

    Δ_cc[1:n_cols+n_rows] = Δ_x_λ
    Δ_cc[n_cols+n_rows+1:n_cols+n_rows+n_low] = @views -(rxs_l+s_l[ilow].*Δ_x_λ[1:n_cols][ilow])./x_minus_lvar
    Δ_cc[n_cols+n_rows+n_low+1:end] = @views (rxs_u+s_u[iupp].*Δ_x_λ[1:n_cols][iupp])./uvar_minus_x 
    return Δ_cc
end

solve_augmented_system_cc! (generic function with 1 method)

In [16]:
function check_frontier!(γf, x, x_minus_lvar, uvar_minus_x, s_l, s_u, Δ, α_pri, α_dual, 
                        lvar, uvar, ilow, iupp, n_low, n_upp, n_rows, n_cols)
    
    x_minus_lvar_sup = @views a_plus_αb(x_minus_lvar, α_pri, Δ[1:n_cols][ilow], n_low)
    uvar_minus_x_sup = @views a_plus_αb(uvar_minus_x, -α_pri, Δ[1:n_cols][iupp], n_upp)
    s_l_sup = @views a_plus_αb(s_l[ilow], α_dual, Δ[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low)
    s_u_sup = @views a_plus_αb(s_u[iupp], α_dual,  Δ[n_rows+n_cols+n_low+1: end], n_upp)
    μ_sup = compute_μ(x_minus_lvar_sup, uvar_minus_x_sup, s_l_sup, s_u_sup, n_low, n_upp)
    f_pri, f_pri_tmp = 0., 0.
    f_dual, f_dual_tmp = 0., 0.
    i_l, i_u = 0, 0
    ilow_pad, iupp_pad = zeros(Int, n_cols), zeros(Int, n_cols)
    ilow_pad[ilow] = 1:n_low
    iupp_pad[iupp] = 1:n_upp
    for i=1:n_cols 
        if ilow_pad[i] != 0 && iupp_pad[i] != 0
            i_l, i_u = ilow_pad[i], iupp_pad[i]
            if (x_minus_lvar_sup[i_l] == 0. || uvar_minus_x_sup[i_u] == 0.)
                f_pri_tmp = (2.0*γf*μ_sup - 
                            x_minus_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]) -
                            uvar_minus_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*
                                (s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if (s_l_sup[i_l] == 0. || uvar_minus_x_sup[i_u] == 0.)
                f_dual_tmp = (2.0*γf*μ_sup - s_l[i_l]*x_minus_lvar_sup[i_l] - s_u[i_u]*uvar_minus_x_sup[i_u]) /
                                (α_dual * (Δ[n_rows+n_cols+i_l]*x_minus_lvar_sup[i_l] +
                                     Δ[n_rows+n_cols+n_low+i_u]*uvar_minus_x_sup[i_u]))
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end  
            end
        elseif ilow_pad[i] != 0 && iupp_pad[i] == 0
            i_l = ilow_pad[i]
            if x_minus_lvar_sup[i_l] == 0. 
                f_pri_tmp = (γf*μ_sup - x_minus_lvar[i_l]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l])) / 
                                (α_pri * Δ[i]*(s_l_sup[i_l]+α_dual*Δ[n_rows+n_cols+i_l]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if s_l_sup[i_l] == 0.
                f_dual_tmp = (γf*μ_sup - s_l[i_l]*x_minus_lvar_sup[i_l] ) /
                                (α_dual * Δ[n_rows+n_cols+i_l]*x_minus_lvar_sup[i_l])
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end
            end 
        elseif ilow_pad[i] == 0 && iupp_pad[i] != 0
            i_u = iupp_pad[i]
            if uvar_minus_x_sup[i_u] == 0.
                f_pri_tmp = (γf*μ_sup - uvar_minus_x[i_u]*(s_u_sup[i_u]+α_dual*Δ[n_rows+n_cols+n_low+i_u])) / 
                                (α_pri * Δ[i]*(-s_u_sup[i_u]-α_dual*Δ[n_rows+n_cols+n_low+i_u]))      
                if f_pri_tmp < f_pri
                    f_pri = f_pri_tmp
                end
            end
            if s_u[i_u] == 0.
                f_dual_tmp = (γf*μ_sup - s_u[i_u]*uvar_minus_x_sup[i_u]) /
                            (α_dual * Δ[n_rows+n_cols+n_low+i_u]*uvar_minus_x_sup[i_u])
                if f_dual_tmp < f_dual
                    f_dual = f_dual_tmp
                end  
            end
        end
    end
    
    α_pri *= max(1.0-γf, f_pri)
    α_dual *= max(1.0-γf, f_dual)
    
    return α_pri, α_dual
end  

check_frontier! (generic function with 1 method)

In [17]:
function a_plus_αb(a, α, b, n)
    # a, b same length, α float
    result = zeros(n)
    for i=1:n
        result[i] = a[i] + α * b[i]
    end
    return result
end

function a_plus_equal_αb(a, α, b, n)
    # a, b same length, α float
    for i=1:n
        a[i] +=  α * b[i]
    end
    return a
end

function get_norm_cols(A, n_cols)
    c = ones(n_cols)
    for j=1:n_cols
        i = A.colptr[j]
        k = A.colptr[j+1] - 1
        c[j] = i <= k ? sqrt(norm(A.nzval[i:k], Inf)) : 1.0
    end
    return c
end
    

function scaling_Ruiz!(A, n_rows, n_cols, ϵ; max_iter = 100)
    AT = sparse(A')
    D1, D2 = I(n_rows), I(n_cols)
    r, c = get_norm_cols(AT, n_rows), get_norm_cols(A, n_cols)
    convergence = maximum(abs.(1.0 .- r)) <= ϵ && maximum(abs.(1.0 .- c)) <= ϵ
    inv_Dr, inv_Dc = inv(Diagonal(r)), inv(Diagonal(c))
    A = rmul!(A, inv_Dc)
    A = lmul!(inv_Dr, A)
    AT = lmul!(inv_Dc, AT)
    AT = rmul!(AT, inv_Dr)
    D1 *= inv_Dr
    D2 *= inv_Dc
    k = 1
    while !convergence && k < max_iter
        r, c = get_norm_cols(AT, n_rows), get_norm_cols(A, n_cols)
        convergence = maximum(abs.(1.0 .- r)) <= ϵ && maximum(abs.(1.0 .- c)) <= ϵ
        inv_Dr, inv_Dc = inv(Diagonal(r)), inv(Diagonal(c))
        A = rmul!(A, inv_Dc)
        A = lmul!(inv_Dr, A)
        AT = lmul!(inv_Dc, AT)
        AT = rmul!(AT, inv_Dr)
        D1 *= inv_Dr
        D2 *= inv_Dc
        k += 1
    end
    return A, D1, D2, inv(D1), inv(D2)
end


scaling_Ruiz! (generic function with 1 method)

In [42]:
function mehrotraPCQuadBounds(mLCP; max_iter=100, ϵ_pdd=1e-8, ϵ_r2=1e-6, ϵ_r1=1e-6,
                              tol_Δx=1e-16, ϵ_μ=0., max_time=60., scaling=true,
                              display=true)
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = mLCP.lvar, mLCP.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp =  [mLCP.ilow; mLCP.irng], [mLCP.iupp; mLCP.irng] # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    M12 = mLCP.M12
    M12rows, M12cols, M12vals = findnz(M12)
    M21 = mLCP.M21
    M21rows, M21cols, M21vals = findnz(M21)
    n_rows, n_cols = size(M21) 
    M11 = mLCP.M11
    M11rows, M11cols, M11vals = findnz(M11)
    M22 = mLCP.M22
    M22rows, M22cols, M22vals = findnz(M22)
    q1 = mLCP.q1
    q2 = mLCP.q2
    c0 = mLCP.c0
    
    
    # init regularization values
    ρ = 1e5*sqrt(eps())

    J_augmcols = vcat(M11cols, M12cols.+n_cols, M21cols, M22cols.+n_cols, 1:n_cols+n_rows)
    J_augmrows = vcat(M11rows, M12rows, M21rows.+n_cols, M22rows.+n_cols, 1:n_cols+n_rows)
    tmp_diag = zeros(n_cols+n_rows)
    J_augmvals = vcat(-M11vals, M12vals, M21vals, M22vals, tmp_diag)
    J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
    
    x, λ, s_l, s_u, J_P = starting_points(M11, M12, M21, q1, q2, lvar, uvar, 
                                               ilow, iupp, mLCP.irng, J_augm, ρ, n_rows, n_cols)

    M11x = M11 * x
    M12λ = M12*λ
    M21x = M21*x
    M22λ = M22*λ
    
    r1 = -M11x + M12λ + s_l - s_u - q1
    r2 = M21x + q2 - M22λ
    
    x_minus_lvar = @views x[ilow] - lvar[ilow]
    uvar_minus_x = @views uvar[iupp] - x[iupp]
    μ = @views compute_μ(x_minus_lvar, uvar_minus_x,
                         s_l[ilow], s_u[iupp],
                         n_low, n_upp)

    k = 0
    Δ_aff = zeros(n_cols+n_rows+n_low+n_upp)
    Δ_cc = zeros(n_cols+n_rows+n_low+n_upp)
    Δ = zeros(n_cols+n_rows+n_low+n_upp)
    Δ_x_λ = zeros(n_cols+n_rows)
    
    # stopping criterion
    xTM11x = x' * M11x
    q1Tx = q1' * x
    pri_obj = xTM11x/2 + q1Tx + c0
    dual_obj = -q2' * λ - xTM11x/2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0
    pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
    max_r1, max_r2 = norm(r1, Inf), norm(r2, Inf)
    optimal = pdd < ϵ_pdd && max_r2 < ϵ_r2 && max_r1 < ϵ_r1

    n_Δx = 0.
    small_Δx, small_μ = false, μ < ϵ_μ
    Δt = time() - start_time
    tired = Δt > max_time

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :max_r2, :max_r1, :n_Δx, :μ],
                         [Int, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
                        hdr_override=Dict(:k=>"Iter", :pri_obj=>"primal", :pdd=>"pdd",
                                          :max_rb=>"r2 cond", :max_rc=>"r1 cond",
                                          :n_Δx=>"‖Δx‖", :μ=>"μ"))
        @info log_row([k, pri_obj, pdd, max_r2, max_r1, n_Δx, μ])
    end

    

    while k<max_iter && !optimal && !tired # && !small_μ && !small_μ

            # Affine scaling direction
        tmp_diag[1:n_cols] .= -ρ
        tmp_diag[n_cols+1:end] .= ρ
        tmp_diag[ilow] .-= @views s_l[ilow] ./ x_minus_lvar
        tmp_diag[iupp] .-= @views s_u[iupp] ./ uvar_minus_x
        J_augmvals[end-n_cols-n_rows+1:end] = tmp_diag

        J_augm = sparse(J_augmrows, J_augmcols, J_augmvals)
        J_fact = lu(J_augm)

        Δ_aff = solve_augmented_system_aff!(J_fact, Δ_aff, Δ_x_λ, r1, r2, x_minus_lvar, uvar_minus_x,
                                            s_l, s_u, ilow, iupp,  n_cols, n_rows, n_low)
        
        α_aff_pri = @views compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = @views compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = @views compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        μ_aff = @views compute_μ(a_plus_αb(x_minus_lvar, α_aff_pri, Δ_aff[1:n_cols][ilow], n_low),
                                 a_plus_αb(uvar_minus_x, -α_aff_pri, Δ_aff[1:n_cols][iupp], n_upp),
                                 a_plus_αb(s_l[ilow], α_aff_dual_final, Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low),
                                 a_plus_αb(s_u[iupp], α_aff_dual_final,  Δ_aff[n_rows+n_cols+n_low+1: end], n_upp),
                                 n_low, n_upp)
        
        σ = (μ_aff / μ)^3

        # corrector and centering step
        Δ_cc = solve_augmented_system_cc!(J_fact, Δ_cc, Δ_x_λ , Δ_aff, σ, μ,x_minus_lvar, uvar_minus_x,
                                          s_l, s_u, ilow, iupp, n_cols, n_rows, n_low)
        Δ = Δ_aff + Δ_cc # final direction
        
        α_pri = @views compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = @views compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = @views compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        α_pri, α_dual_final = check_frontier!(1.0e-3, x, x_minus_lvar, uvar_minus_x, s_l, s_u,
                                              Δ, α_pri, α_dual_final, lvar, uvar, ilow, iupp,
                                              n_low, n_upp, n_rows, n_cols)

        # new parameters
        x = @views a_plus_equal_αb(x, α_pri, Δ[1:n_cols], n_cols)
        λ = @views a_plus_equal_αb(λ, α_dual_final,
                                   Δ[n_cols+1: n_rows+n_cols], n_rows)
        s_l[ilow] = @views a_plus_equal_αb(s_l[ilow], α_dual_final,
                                           Δ[n_rows+n_cols+1: n_rows+n_cols+n_low], n_low)
        s_u[iupp] = @views a_plus_equal_αb(s_u[iupp], α_dual_final,
                                           Δ[n_rows+n_cols+n_low+1: end], n_upp)
        n_Δx = @views α_pri * norm(Δ[1:n_cols])
        x_minus_lvar = @views x[ilow] - lvar[ilow]
        uvar_minus_x = @views uvar[iupp] - x[iupp]

        μ = @views compute_μ(x_minus_lvar, uvar_minus_x,
                             s_l[ilow], s_u[iupp],
                             n_low, n_upp)

        M11x = mul!(M11x, M11, x)
        xTM11x = x' * M11x 
        q1Tx = q1' * x
        M12λ = mul!(M12λ, M12, λ)
        M21x = mul!(M21x, M21, x)
        M22λ = mul!(M22λ, M22, λ)
        pri_obj = xTM11x/2 + q1Tx + c0  # no sense for a mLCP?
        dual_obj = -q2' * λ - xTM11x/2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0

        r1 = -M11x + M12λ + s_l - s_u - q1
        r2 = M21x + q2 - M22λ
        
        # update stopping criterion values:
        
        pdd = abs(pri_obj - dual_obj ) / (1. + abs(pri_obj))
        max_r1, max_r2 = norm(r1, Inf), norm(r2, Inf)
        optimal = pdd < ϵ_pdd && max_r2 < ϵ_r2 && max_r1 < ϵ_r1
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        
        if ρ >= 1*sqrt(eps())
            ρ /= 10
        end
        
        Δt = time() - start_time
        tired = Δt > max_time

        if display == true
            @info log_row([k, pri_obj, pdd, max_r2, max_r1, n_Δx, μ])
        end
        

    end
    
    if display == true
        criteria = [k >= max_iter,  optimal, small_Δx, small_μ]
        criteria_names = ["reached max_iter",  "optimal", 
            "n_Δx <= small_Δx", "μ <= ϵ_μ"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    else
        status = :acceptable
    end
    
"""    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj , 
                                  dual_feas = cond_r1 * n_1_p1, 
                                  primal_feas = cond_r2 * n_2_p1,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)"""
    stats = Dict()
    stats["objective"] = pri_obj
    stats["dual_feas"] = max_r1
    stats["primal_feas"] = max_r2
    stats["iter"] = k
    stats["elapsed_time"] = elapsed_time
    stats["solution"] = x
    return stats
end


mehrotraPCQuadBounds (generic function with 1 method)

In [45]:
# probleme1
Q = [6. 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1. 0 1
    0 1 1]
b = [0.; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
ilow = findall((x -> x!=-Inf), lvar)  
iupp = findall((x -> x!=Inf), uvar)
irng = []
lcon = b
ucon = b

n_x, n_λ = 3, 2


x01 = [1.; 2.; 3.];


mLCP1 = mLCP(c0, c, -b, sparse(Q), sparse(A'), sparse(A), sparse([0. 0; 0 0]), lvar, uvar, ilow, iupp, irng )

mLCP(0.0, [-8, -3, -3], [-0.0, -3.0], 
  [1, 1]  =  6.0
  [2, 1]  =  2.0
  [3, 1]  =  1.0
  [1, 2]  =  2.0
  [2, 2]  =  5.0
  [3, 2]  =  2.0
  [1, 3]  =  1.0
  [2, 3]  =  2.0
  [3, 3]  =  4.0, 
  [1, 1]  =  1.0
  [3, 1]  =  1.0
  [2, 2]  =  1.0
  [3, 2]  =  1.0, 
  [1, 1]  =  1.0
  [2, 2]  =  1.0
  [1, 3]  =  1.0
  [2, 3]  =  1.0, 2×2 SparseMatrixCSC{Float64,Int64} with 0 stored entries, [0, 0, 0], [Inf, Inf, Inf], [1, 2, 3], Int64[], Any[])

In [46]:
stats_mpc1 =  mehrotraPCQuadBounds(mLCP1)


 stopping criterion = ["optimal"]


┌ Info:   Iter    primal       pdd    max_r2    max_r1      ‖Δx‖         μ  
└ @ Main In[42]:75
┌ Info:  0.0e+00   1.4e+02   2.1e+00   6.2e+00   1.2e+01   0.0e+00   4.6e+01
└ @ Main In[42]:80
┌ Info:  1.0e+00   1.8e+01   1.3e+00   9.0e-01   3.4e+00   4.4e+00   6.4e+00
└ @ Main In[42]:174
┌ Info:  2.0e+00   1.4e+01   9.9e-03   8.5e-03   3.0e-02   9.0e-01   6.2e-02
└ @ Main In[42]:174
┌ Info:  3.0e+00   1.4e+01   2.0e-05   1.7e-05   6.0e-05   8.6e-03   1.2e-04
└ @ Main In[42]:174
┌ Info:  4.0e+00   1.4e+01   3.9e-08   3.4e-08   1.2e-07   1.7e-05   2.5e-07
└ @ Main In[42]:174
┌ Info:  5.0e+00   1.4e+01   7.8e-11   6.8e-11   2.4e-10   3.4e-08   5.0e-10
└ @ Main In[42]:174


Dict{Any,Any} with 6 entries:
  "dual_feas"    => 2.40114e-10
  "primal_feas"  => 6.80144e-11
  "objective"    => 13.5
  "elapsed_time" => 0.0209999
  "solution"     => [5.34687e-11, 3.0, 1.45457e-11]
  "iter"         => 5

In [23]:
mLCP1.lvar

3-element Array{Int64,1}:
 0
 0
 0

In [44]:
typeof(Q)

Array{Float64,2}